# Import Necessary Libraries

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [29]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [30]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [31]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over XGBoost model

In [ ]:
parameters = {
    'max_depth': [8, 10],
    'n_estimators': [14, 16, 18],
    'learning_rate': [0.2, 0.25, 0.3],
    'gamma':[1.0, 1.5, 2.0]
}
xgb = XGBClassifier(objective='multi:softmax', tree_method='gpu_hist', gpu_id=0)
grid = GridSearchCV(xgb, parameters, cv=2, scoring='accuracy', verbose=2)
grid.fit(X_train, Y_train)

Fitting 2 folds for each of 54 candidates, totalling 108 fits
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=14 ......


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=14, total=   3.1s
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=14 ......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s


[CV]  gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=14, total=   3.1s
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=16 ......
[CV]  gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=16, total=   3.0s
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=16 ......
[CV]  gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=16, total=   3.1s
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=18 ......
[CV]  gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=18, total=   3.3s
[CV] gamma=1.0, learning_rate=0.2, max_depth=8, n_estimators=18 ......


In [ ]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

In [ ]:
clf = best_estimator
clf.fit(X_train,Y_train)

In [ ]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

# Saving model

In [41]:
pickle.dump(clf, open(os.path.join("..", "IPD", "XGBoost_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
